<div style="align: center; margin: 0; padding: 0; height: 250px;">
    <br>
    <img src="https://www.nyc.gov/assets/tlc/images/content/hero/MRP-Closing-Week.jpg" style="display:block; margin:auto; width:65%; height:100%;">
</div><br><br> 

<div style="letter-spacing:normal; opacity:1.;">
<!--   https://xkcd.com/color/rgb/   -->
  <p style="text-align:center; background-color: lightsalmon; color: Jaguar; border-radius:10px; font-family:monospace; 
            line-height:1.4; font-size:32px; font-weight:bold; text-transform: uppercase; padding: 9px;">
            <strong>TLC Trip Record Data</strong></p>  
  
  <p style="text-align:center; background-color:romance; color: Jaguar; border-radius:10px; font-family:monospace; 
            line-height:1.4; font-size:22px; font-weight:normal; text-transform: capitalize; padding: 5px;"
     >Machine Learning Module: MONITORING - Ride Duration Prediction<br>using Regression Analysis<br></p><br>
    
  <div style="align: center;">
  <table style="text-align: center; background-color: romance; color: Jaguar; border-radius: 10px; font-family: monospace;
                  line-height:1.4; font-size: 21px; font-weight: normal; text-transform: capitalize; padding: 5px; 
                  margin: 0 auto;">
    <tr><td style="text-align: left; padding-left: 0px;"
            > MONITORING <span style="font-size: 16px;">(to ensure that deployed models remain accurate,<br>reliable, and aligned with business objectives over time)</span></td></tr>
    <tr><td style="text-align: left; padding-left: 0px;"
            > MLOps <span style="font-size: 16px;">(CI/CD, Model Versioning, Monitoring, Automated Retraining,<br>Security, Scalability, Collaboration)</span></td></tr>
  </table>
  </div>

</div>

[Introduction to MLOps Monitoring](https://github.com/dimzachar/mlops-zoomcamp/blob/master/notes/Week_5/intro.md)

**Dataset Info**


**Context**

Yellow and green taxi trip records include fields capturing pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts. The data used in the attached datasets were collected and provided to the NYC Taxi and Limousine Commission (TLC) by technology providers authorized under the Taxicab & Livery Passenger Enhancement Programs (TPEP/LPEP). The trip data was not created by the TLC, and TLC makes no representations as to the accuracy of these data.

For-Hire Vehicle (“FHV”) trip records include fields capturing the dispatching base license number and the pick-up date, time, and taxi zone location ID (shape file below). These records are generated from the FHV Trip Record submissions made by bases. Note: The TLC publishes base trip record data as submitted by the bases, and we cannot guarantee or confirm their accuracy or completeness. Therefore, this may not represent the total amount of trips dispatched by all TLC-licensed bases. The TLC performs routine reviews of the records and takes enforcement actions when necessary to ensure, to the extent possible, complete and accurate information.


**ATTENTION!**

On 05/13/2022, we are making the following changes to trip record files:

- All files will be stored in the PARQUET format. Please see the ‘Working With PARQUET Format’ under the Data Dictionaries and MetaData section.
- Trip data will be published monthly (with two months delay) instead of bi-annually.
- HVFHV files will now include 17 more columns (please see High Volume FHV Trips Dictionary for details). Additional columns will be added to the old files as well. The earliest date to include additional columns: February 2019.
- Yellow trip data will now include 1 additional column (‘airport_fee’, please see Yellow Trips Dictionary for details). The additional column will be added to the old files as well. The earliest date to include the additional column: January 2011.


**Download the data for March 2023**

Dataset: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page


**Data Dictionaries and MetaData**

- We'll use the same `NYC taxi dataset`, but instead of "Yellow Taxi Trip Records", we'll use `"Green Taxi Trip Records"`.

> `Green Trips Data Dictionary`: https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_green.pdf

**TASK**


The goal of this homework is to familiarize users with monitoring for ML batch services, using PostgreSQL database to store metrics and Grafana to visualize them. Download the March 2023 Green Taxi data.

[homework](https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2023/05-monitoring/homework.md)


**Note**: Model Monitoring: Model monitoring is the process of continuously tracking and evaluating the performance and behavior of machine learning models deployed in production. It involves collecting relevant data, assessing model outputs, detecting anomalies or deviations, and triggering alerts or actions when necessary. The goal of model monitoring is to ensure that deployed models remain accurate, reliable, and aligned with business objectives over time.


**Table of Content**


1. Import Libraries and Ingest Data 
2. Docker Compose Configuration File
    - Create a Docker Compose File
    - Create Grafana Data Source Configuration
    - Build and Run Docker Compose<br>
3. Baseline Model nyc_taxi Data
    - Q1. Prepare the dataset
    - Q2. Metric
    - Q3. Prefect flow
    - Q4. Monitoring
    - Q5. Dashboard<br>
    

<div style="letter-spacing:normal; opacity:1.;">
  <h1 style="text-align:center; background-color: lightsalmon; color: Jaguar; border-radius:10px; font-family:monospace; border-radius:20px;
            line-height:1.4; font-size:32px; font-weight:bold; text-transform: uppercase; padding: 9px;">
            <strong>1. Import Libraries & Ingest Data</strong></h1>   
</div>

> ⚠️ Not Recommended conda `base` env, work on `venv`

- https://docs.conda.io/projects/conda/en/4.6.0/_downloads/52a95608c49671267e40c689e0bc00ca/conda-cheatsheet.pdf

```
pip freeze > requirements.txt
conda list -e > requirements.txt

# new conda virtual environment
conda create --name "monitoring-ops" python=3.10 jupyter -y
conda activate "monitoring-ops"

# install all package dependencies
pip install -r requirements.txt
conda install -c conda-forge --file=requirements.txt      # mostly not work
conda install -c conda-forge pandas==2.0.2 -q -y

# if The environment is inconsistent, try below
conda update -n base -c defaults conda --force-reinstall
conda install anaconda --force-reinstall

```

**You must use the `--no-deps` option in the pip install command in order to avoid bundling dependencies into your conda-package.**

If you run pip install without the `--no-deps` option, pip will often install dependencies in your conda recipe and those dependencies will become part of your package. This wastes space in the package and `increases the risk of file overlap`, file clobbering, and broken packages.

There might be cases where you want to install a package directly from a local directory or a specific location, without relying on the package indexes. In such situations, you can use the `--no-index` option to tell pip not to look for the package in any indexes.

```
- command1 & command2  # runs simultaneously
- command1 ; command2  # runs sequentially
- command1 && command2 # runs sequentially, runs command2 only if command1 succeeds
- command1 || command2 # runs sequentially, runs command2 only if command1 fails
```

In [1]:
# # %%capture cap --no-stderr  # capture outputs  # cap.show()
# !cat /etc/os-release
# !grep -E -w 'VERSION|NAME|PRETTY_NAME' /etc/os-release

In [2]:
# check enviroment
# !conda env list
# !conda info -e
# !conda info | grep 'active env'

In [3]:
%%writefile requirements.txt 
# To get started with MLflow you'll need to install the appropriate Python package.

# for parquet file
pyarrow==11.0.0
fastparquet==2023.4.0

pandas==2.0.2
matplotlib==3.7.1
orjson==3.8.8          # orjson is a fast, correct JSON library
tqdm==4.65.0
# requests==2.29.0

# ML Model packages
scikit-learn==1.2.2

# MLOPS packages
mlflow==2.3.1
prefect==2.10.18
prefect-email==0.2.2

# MLOPS Cloud packages
boto3~=1.24.28
prefect-aws==0.3.4

# Monitoring
joblib==1.2.0
evidently==0.3.3
psycopg==3.1.9
# psycopg[binary]==3.1.9

# Optionally
pipenv~=2022.7.4
jupyter
ipykernel
ipywidgets

Overwriting requirements.txt


In [4]:
import os, sys, platform, IPython.display

# !{sys.executable} -m pip install -Uq -r requirements.txt  #  --no-deps --no-cache-dir --force-reinstall --no-index
!jupyter nbextension enable --py widgetsnbextension

# IPython.display.clear_output()
print("Python  :", sys.version)
print("Platform:", platform.system(), platform.platform())
print("Actv Env:", os.environ['CONDA_DEFAULT_ENV'])

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Python  : 3.10.0 (default, Mar  3 2022, 09:58:08) [GCC 7.5.0]
Platform: Linux Linux-5.15.90.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
Actv Env: monitoring-ops


In [5]:
import warnings
from numba import NumbaDeprecationWarning
# Suppress NumbaDeprecationWarning in umap.distances module
warnings.filterwarnings("ignore", category=NumbaDeprecationWarning)

import os
# TensorFlow to only display error messages 
# and suppress warnings and informational messages.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import scipy
from scipy.stats import stats

import os
import sys
import boto3
import click
import pickle
import datetime
# import pathlib
# import argparse
import requests
# import urllib.request
from glob import glob

# Import joblib for model persistence and tqdm for progress bars
from joblib import load, dump
# from tqdm import tqdm           # console-based
# from tqdm.notebook import tqdm  # jupyter-based
from tqdm.auto import tqdm        # automatically selects
# tqdm._instances.clear()

# Import sklearn for machine learning
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# import mlflow
# import wandb
# import prefect
# from prefect import task, flow, Flow
# from prefect.tasks import task_input_hash
# from prefect.artifacts import create_markdown_artifact

# Import Evidently for data drift and model performance monitoring
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric
from evidently.metrics import DatasetMissingValuesMetric, ColumnQuantileMetric

# memory management performs garbage collection 
import gc
gc.collect()

0

<div style="letter-spacing:normal; opacity:1.;">
  <h1 style="text-align:center; background-color: lightsalmon; color: Jaguar; border-radius:10px; font-family:monospace; border-radius:20px;
            line-height:1.4; font-size:32px; font-weight:bold; text-transform: uppercase; padding: 9px;">
            <strong>2. Recognizing and Understanding Data</strong></h1>   
</div>

## Ingest Data [wget](https://linuxways.net/centos/linux-wget-command-with-examples/) or [curl](https://daniel.haxx.se/blog/2020/09/10/store-the-curl-output-over-there/)

In [6]:
# "Green Taxi Trip Records" Download the data for January, February and March 2022
# !wget -q -N -P "./data" https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet

In [7]:
glob(f'./data/*.parquet')

['./data/green_tripdata_2023-03.parquet', './data/reference.parquet']

In [8]:
import os
# Get the current working directory
# current_dir = os.getcwd()

# Create a new directory for storing MLflow data
os.makedirs('./config', exist_ok=True)
os.makedirs('./pycode', exist_ok=True)
# os.makedirs('./data', exist_ok=True)
# os.makedirs('./output', exist_ok=True)
# os.makedirs('./models', exist_ok=True)

# Docker Compose Configuration File

A Docker Compose file is used to define and manage a multi-container Docker application. It's a YAML file that contains all the necessary configurations to run the application. The services defined in this file are typically used in a development environment for testing or in a production environment for deployment.

## Create a Docker Compose File

- In your project directory, create a file named docker-compose.yml

Next, we'll configure the Docker Compose file, including the specification of volumes for Grafana. We'll add a PostgreSQL database, an Adminer for managing the database content, and Grafana for dashboarding.

Here's an example of docker-compose.yml:

```sh
touch docker-compose.yml
```

In [9]:
%%writefile docker-compose.yml

# Specifies the Docker Compose file version
version: '3.7'

# Declares volumes that can be used by services in the Docker Compose file
volumes: 
  # Declares a volume named grafana_data
  grafana_data: {}

# Defines networks that can be used by services in the Docker Compose file
networks:
  # Declares a network named front-tier
  front-tier:
  # Declares a network named back-tier
  back-tier:

# Defines the services that make up your app
services:
  # Defines a service named db
  db:
    # Specifies the Docker image to use for this service
    image: postgres
    # Ensures that the service is always restarted if it stops
    restart: always
    # Sets environment variables for the service
    environment:
      # Sets the password for the Postgres database
      POSTGRES_PASSWORD: example
    # Maps ports between the host and the container
    ports:
      - "5432:5432"
    # Specifies the networks that this service is part of
    networks:
      - back-tier

  # Defines a service named adminer
  adminer:
    image: adminer
    restart: always
    ports:
      - "8080:8080"
    networks:
      - back-tier
      - front-tier  

  # Defines a service named grafana
  grafana:
    image: grafana/grafana
    # Sets the user ID under which the service will run
    user: "472"
    ports:
      - "3000:3000"
    # Maps local directories or files to directories inside the container
    volumes:
      # Maps a local file to a file inside the container, and makes it read-only
      - ./config/grafana_datasources.yaml:/etc/grafana/provisioning/datasources/datasource.yaml:ro
      - ./config/grafana_dashboards.yaml:/etc/grafana/provisioning/dashboards/dashboards.yaml:ro
      - ./dashboards:/opt/grafana/dashboards
    networks:
      - back-tier
      - front-tier
    restart: always

Overwriting docker-compose.yml


## Create Grafana Data Source Configuration

- The default username and password for Grafana webpage is admin / admin .

We'll create a Grafana data source configuration file. It's used to define the data sources that Grafana should connect to. A data source in Grafana represents a back-end database, such as PostgreSQL, MySQL, InfluxDB, or a myriad of other data storage systems. In your project directory, create a new directory named config and a file within it named grafana_datasources.yaml:

In [10]:
%%writefile config/grafana_datasources.yaml

# Specifies the version used in this configuration file
apiVersion: 1

# Defines the data sources that Grafana should connect to
# list of datasources to insert/update
# available in the database
datasources:
  # Defines a data source
  - 
    # Specifies the name of the data source
    name: PostgreSQL
    # Specifies the type of the data source
    type: postgres
    # Specifies how Grafana should access the data source
    # The 'proxy' mode means that all requests are proxied via the Grafana backend/server
    access: proxy
    # Specifies the URL (including the port number) of the data source
    url: db.:5432
    # Specifies the name of the database that Grafana should connect to
    database: test
    # Specifies the username that Grafana should use to connect to the database
    user: postgres
    # Specifies secure data like passwords
    secureJsonData:
      # Sets the password for the database connection
      password: 'example'
    # Specifies additional JSON data for the data source configuration
    jsonData:
      # Disables SSL mode for the database connection
      sslmode: 'disable'

Overwriting config/grafana_datasources.yaml


In [11]:
%%writefile config/grafana_dashboards.yaml

# config file version
apiVersion: 1

providers:
  # <string> an unique provider name. Required
  - name: 'Evidently Dashboards'
    # <int> Org id. Default to 1
    orgId: 1
    # <string> name of the dashboard folder.
    folder: ''
    # <string> folder UID. will be automatically generated if not specified
    folderUid: ''
    # <string> provider type. Default to 'file'
    type: file
    # <bool> disable dashboard deletion
    disableDeletion: false
    # <int> how often Grafana will scan for changed dashboards
    updateIntervalSeconds: 10
    # <bool> allow updating provisioned dashboards from the UI
    allowUiUpdates: false
    options:
      # <string, required> path to dashboard files on disk. Required when using the 'file' type
      path: /opt/grafana/dashboards
      # <bool> use folder names from filesystem to create folders in Grafana
      foldersFromFilesStructure: true

Overwriting config/grafana_dashboards.yaml


## Build and Run Docker Compose

Finally, we'll build and run our Docker Compose configuration. In your terminal, navigate to the directory containing docker-compose.yml and run:

In [12]:
# docker-compose up --build

You should see that all your containers are successfully created. You can verify this by accessing Grafana and Adminer through your browser at:
- [localhost:3000](http://localhost:3000) and 
- [localhost:8080](http://localhost:8080), respectively.

That's it! You've now set up an environment for MLOps using Docker Compose. In the next part, we'll load some data and start implementing monitoring for our service.

Then open NEW terminal activate CONDA enviroment in terminal, navigate to the directory containing docker-compose.yml and run below code stop docker container:

In [13]:
# docker-compose down

# Baseline Model nyc_taxi Data

## Q1. Prepare the dataset

Start with `baseline_model_nyc_taxi_data.ipynb`. Download the March 2023 Green Taxi data. We will use this data to simulate a production usage of a taxi trip duration prediction service.

**What is the shape of the downloaded data? How many rows are there?**

* 72044
* 78537 
* 62495
* 54396

**march_data.shape**

(72044, 20)

In [14]:
# print(github_link.replace("github.com", "raw.githubusercontent.com").replace("/blob/", "/"))

# !wget -Nq "https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/05-monitoring/baseline_model_nyc_taxi_data.ipynb"
# !wget -Nq "https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/05-monitoring/debugging_nyc_taxi_data.ipynb"

## Q2. Metric

Let's expand the number of data quality metrics we’d like to monitor! Please add one metric of your choice and a quantile value for the `"fare_amount"` column (`quantile=0.5`).

Hint: explore evidently metric `ColumnQuantileMetric` (from `evidently.metrics import ColumnQuantileMetric`) 

What metric did you choose?

From the provided metrics, the metric **chosen** for evaluation is the **ColumnDriftMetric** for the 'prediction' column. It calculates the drift score and detects whether drift is present. The drift score in this case is 0.0219, and the 'drift_detected' field is False, indicating that no drift is detected in the 'prediction' column.

```py
# prediction drift
result['metrics'][0]['result']['drift_score']

# 0.02191274473735342
```

## Q3. Prefect flow 

Let’s update prefect tasks by giving them nice meaningful names, specifying a number of delays and retries.

Hint: use `evidently_metrics_calculation.py` script as a starting point to implement your solution. Check the  prefect docs to check task parameters.

**What is the correct way of doing that?**
- https://docs.prefect.io/2.10.18/api-ref/prefect/tasks/#prefect.tasks.task

* `@task(retries_num=2, retry_seconds=5, task_name="calculate metrics")`
* `@task(retries_num=2, retry_delay_seconds=5, name="calculate metrics")`
* `@task(retries=2, retry_seconds=5, task_name="calculate metrics")`
* `@task(retries=2, retry_delay_seconds=5, name="calculate metrics")`

In [15]:
# !wget -Nq "https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/05-monitoring/dummy_metrics_calculation.py"
# !wget -Nq "https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/05-monitoring/evidently_metrics_calculation.py"

In [16]:
%%writefile pycode/dummy_metrics_calculation.py

import io
import uuid
import pytz
import time
import datetime
import random
import psycopg
import logging 
import pandas as pd

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s]: %(message)s")

SEND_TIMEOUT = 10
rand = random.Random()

create_table_statement = """
drop table if exists dummy_metrics;
create table dummy_metrics(
	timestamp timestamp,
	value1 integer,
	value2 varchar,
	value3 float
)
"""


def prep_db():
	with psycopg.connect("host=localhost port=5432 user=postgres password=example", autocommit=True) as conn:
		res = conn.execute("SELECT 1 FROM pg_database WHERE datname='test'")
		if len(res.fetchall()) == 0:
			conn.execute("create database test;")
		with psycopg.connect("host=localhost port=5432 dbname=test user=postgres password=example") as conn:
			conn.execute(create_table_statement)


def calculate_dummy_metrics_postgresql(curr):
	value1 = rand.randint(0, 1000)
	value2 = str(uuid.uuid4())
	value3 = rand.random()

	curr.execute(
		"insert into dummy_metrics(timestamp, value1, value2, value3) values (%s, %s, %s, %s)",
		(datetime.datetime.now(pytz.timezone('Europe/London')), value1, value2, value3)
	)


def main():
	prep_db()
	last_send = datetime.datetime.now() - datetime.timedelta(seconds=10)
	with psycopg.connect("host=localhost port=5432 dbname=test user=postgres password=example", autocommit=True) as conn:
		for i in range(0, 100):
			with conn.cursor() as curr:
				calculate_dummy_metrics_postgresql(curr)

			new_send = datetime.datetime.now()
			seconds_elapsed = (new_send - last_send).total_seconds()
			if seconds_elapsed < SEND_TIMEOUT:
				time.sleep(SEND_TIMEOUT - seconds_elapsed)
			while last_send < new_send:
				last_send = last_send + datetime.timedelta(seconds=10)
			logging.info("data sent")
			

if __name__ == '__main__':
	main()

Overwriting pycode/dummy_metrics_calculation.py


In [21]:
%%writefile pycode/evidently_metrics_calculation.py

import warnings
from numba import NumbaDeprecationWarning
# Suppress NumbaDeprecationWarning in umap.distances module
warnings.filterwarnings("ignore", category=NumbaDeprecationWarning)

import os
# TensorFlow to only display error messages 
# and suppress warnings and informational messages.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import datetime
import time
import random
import logging 
import uuid
import pytz
import pandas as pd
import io
import psycopg
import joblib

from prefect import task, flow

# Import Evidently for data drift and model performance monitoring
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, ColumnQuantileMetric
from evidently.metrics import DatasetDriftMetric, DatasetMissingValuesMetric

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s]: %(message)s")

SEND_TIMEOUT = 10
rand = random.Random()

create_table_statement = """
drop table if exists evidently_metrics_calculation;
create table evidently_metrics_calculation(
	timestamp timestamp,
	prediction_drift float,
	num_drifted_columns integer,
	share_missing_values float,
	column_quantile_values float
);
"""


with open('models/lin_reg.bin', 'rb') as f_in:
	model = joblib.load(f_in)	

begin = datetime.datetime(2023, 3, 1, 0, 0)
raw_data = pd.read_parquet('data/green_tripdata_2023-03.parquet')
reference_data = pd.read_parquet('data/reference.parquet')

# Define the column mapping for the Evidently report
num_features = ['passenger_count', 'trip_distance', 'fare_amount', 'total_amount']
cat_features = ['PULocationID', 'DOLocationID']
column_mapping = ColumnMapping(
    prediction='prediction',
    numerical_features=num_features,
    categorical_features=cat_features,
    target=None
)

# Create a list of metrics to include in the report
metrics = [
	ColumnDriftMetric(column_name='prediction'),
	DatasetDriftMetric(),
	DatasetMissingValuesMetric(),    
	# Add the ColumnQuantileMetric for the 'fare_amount' column
	ColumnQuantileMetric(column_name='fare_amount', quantile=0.5)
]

# Create the report with the metrics
report = Report(metrics=metrics)


@task(name="Preprocces Database", log_prints=None)
def prep_db():
	with psycopg.connect("host=localhost port=5432 user=postgres password=example", autocommit=True) as conn:
		res = conn.execute("SELECT 1 FROM pg_database WHERE datname='test'")
		if len(res.fetchall()) == 0:
			conn.execute("create database test;")
		with psycopg.connect("host=localhost port=5432 dbname=test user=postgres password=example") as conn:
			conn.execute(create_table_statement)


@task(name="Calculate Metrics", retries=2, retry_delay_seconds=5, log_prints=None)
def calculate_metrics_postgresql(curr, i):
	current_data = raw_data[(raw_data.lpep_pickup_datetime >= (begin + datetime.timedelta(i))) &
		(raw_data.lpep_pickup_datetime < (begin + datetime.timedelta(i + 1)))]

	#current_data.fillna(0, inplace=True)
	current_data['prediction'] = model.predict(current_data[num_features + cat_features].fillna(0))

	report.run(
		reference_data = reference_data, 
		current_data = current_data,
		column_mapping=column_mapping
	)
	result = report.as_dict()

	prediction_drift = result['metrics'][0]['result']['drift_score']
	num_drifted_columns = result['metrics'][1]['result']['number_of_drifted_columns']
	share_missing_values = result['metrics'][2]['result']['current']['share_of_missing_values']
	column_quantile_values = result['metrics'][3]['result']['current']['value']

	curr.execute(
		"insert into evidently_metrics_calculation(timestamp, prediction_drift, num_drifted_columns, share_missing_values, column_quantile_values) values (%s, %s, %s, %s, %s)",
		(begin + datetime.timedelta(i), prediction_drift, num_drifted_columns, share_missing_values, column_quantile_values)
	)


@flow(name="Batch Monitoring Backfill", log_prints=None)
def batch_monitoring_backfill():
	prep_db()
	last_send = datetime.datetime.now() - datetime.timedelta(seconds=10)
	with psycopg.connect("host=localhost port=5432 dbname=test user=postgres password=example", autocommit=True) as conn:
		for i in range(0, 27):
			with conn.cursor() as curr:
				calculate_metrics_postgresql(curr, i)

			new_send = datetime.datetime.now()
			seconds_elapsed = (new_send - last_send).total_seconds()
			if seconds_elapsed < SEND_TIMEOUT:
				time.sleep(SEND_TIMEOUT - seconds_elapsed)
			while last_send < new_send:
				last_send = last_send + datetime.timedelta(seconds=10)
			logging.info("data sent")


if __name__ == '__main__':
	batch_monitoring_backfill()

Overwriting pycode/evidently_metrics_calculation.py


so we edit dummy_metrics_calculation.py and evidently_metrics_calculation.py

- Start Prefect server
```sh
prefect server start

```
- start again docker compose, if you want to rebuild your services use --build:
```sh
docker-compose up --build
```

- Let's test our script RUN the script and go to Grafana and Adminer through your browser at:
    - [localhost:3000](http://localhost:3000) and 
    - [localhost:8080](http://localhost:8080), respectively.

In [20]:
# python pycode/dummy_metrics_calculation.py
# python pycode/evidently_metrics_calculation.py

## Q4. Monitoring

Let’s start monitoring. Run expanded monitoring for a new batch of data (March 2023). 

What is the maximum value of metric `quantile = 0.5` on th `"fare_amount"` column during March 2023 (calculated daily)?

* 10
* 12.5
* 14
* 14.8

In [22]:
pd.read_csv('data/evidently_metrics_calculation.csv')

,timestamp,prediction_drift,num_drifted_columns,share_missing_values,column_quantile_values
0,2023-03-01 00:00:00,0.315424,4,0.064327,12.80
1,2023-03-02 00:00:00,2.080613,4,0.065870,13.50
2,2023-03-03 00:00:00,1.743415,4,0.066032,13.50
3,2023-03-04 00:00:00,0.306647,4,0.066331,13.50
4,2023-03-05 00:00:00,0.874525,4,0.064108,13.50
5,2023-03-06 00:00:00,1.054192,4,0.064483,13.05
6,2023-03-07 00:00:00,0.811607,4,0.061817,12.80
7,2023-03-08 00:00:00,0.073897,2,0.054498,12.80
8,2023-03-09 00:00:00,0.132309,6,0.067361,13.50
9,2023-03-10 00:00:00,0.118955,4,0.070164,13.50


In [23]:
pd.read_csv('data/evidently_metrics_calculation.csv')['column_quantile_values'].max()

14.0

## Q5. Dashboard


Finally, let’s add panels with new added metrics to the dashboard. After we customize the  dashboard lets save a dashboard config, so that we can access it later. Hint: click on “Save dashboard” to access JSON configuration of the dashboard. This configuration should be saved locally.

Where to place a dashboard config file?

* `project_folder` (05-monitoring)
* `project_folder/config`  (05-monitoring/config)
* `project_folder/dashboards`  (05-monitoring/dashboards)
* `project_folder/data`  (05-monitoring/data)

In [ ]:
# project_folder/dashboards (05-monitoring/dashboards)

## Submit the results

* Submit your results here: https://forms.gle/PJaYeWsnWShAEBF79
* You can submit your solution multiple times. In this case, only the last submission will be used
* If your answer doesn't match options exactly, select the closest one


## Deadline

The deadline for submitting is 7 July (Friday), 23:00 CEST (Berlin time). 

After that, the form will be closed.

# End of The Project